<a href="https://colab.research.google.com/github/SAGARMOHANTY98/SAGAR/blob/main/bobbin_winding_planning_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
from fuzzywuzzy import process

def calculate_bobbin_plan(frame_file, plan_file):
    try:
        # Read input files
        df_frame = pd.read_excel('/content/bw plan (1).xlsx')
        df_plan = pd.read_excel('/content/material wise planning.xlsx')

        # 1. Calculate Frame Requirements using Production Order No.
        # Ensure 'Prod. Order No.' is treated as string to avoid potential merge issues
        df_frame['Prod. Order No.'] = df_frame['Prod. Order No.'].astype(str)

        frame_info = (df_frame.groupby('Prod. Order No.')['Item Type']
                      .nunique()
                      .reset_index()
                      .rename(columns={'Item Type': 'Unique Types'}))
        frame_info['Frame Needed'] = frame_info['Unique Types'].apply(lambda x: 2 if x > 1 else 1)

        # Count single and double frame items
        frame_counts = frame_info['Frame Needed'].value_counts()
        single_frame_count = frame_counts.get(1, 0)
        double_frame_count = frame_counts.get(2, 0)

        # Keep only necessary columns for merging
        frame_info_merge = df_frame[['Prod. Order No.', 'Description']].drop_duplicates()
        frame_info = pd.merge(frame_info,
                              frame_info_merge,
                              on='Prod. Order No.',
                              how='left') # Use left merge to keep all production orders

        # 2. Clean and Match Descriptions
        def clean_description(desc):
            if not isinstance(desc, str):
                return desc
            return desc.rsplit('-', 1)[0].strip() if '-' in desc else desc.strip()

        df_plan['Clean Description'] = df_plan['Item Description'].apply(clean_description)

        # Fuzzy matching with threshold
        frame_descriptions = frame_info['Description'].dropna().unique()
        def get_matched_description(clean_desc):
            if pd.notna(clean_desc) and len(frame_descriptions) > 0:
                match = process.extractOne(clean_desc, frame_descriptions, score_cutoff=80)
                return match[0] if match else None
            return None

        df_plan['Matched Description'] = df_plan['Clean Description'].apply(get_matched_description)

        # 3. Merge Data
        # Ensure the merge key 'Matched Description' and 'Description' exist and are strings
        merged = pd.merge(
            df_plan,
            frame_info[['Prod. Order No.', 'Description', 'Frame Needed']].drop_duplicates(), # Select necessary columns from frame_info
            left_on='Matched Description',
            right_on='Description',
            how='left' # Use left merge to keep all plan items
        )

        # 4. Calculate Metrics
        # Stock
        stock_col = next((col for col in ['Total Stock', 'Stock'] if col in merged.columns), None)
        merged['Stock'] = pd.to_numeric(merged[stock_col], errors='coerce').fillna(0) if stock_col else 0

        # Planned KG
        plan_cols = [col for col in ["Over_Due", "Current Week", "Column1", "Week 2"] if col in merged.columns]
        merged['Planned KG'] = merged[plan_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1).fillna(0)

        # Stock Balance and Order Required
        merged['Stock Balance'] = merged['Stock'] - merged['Planned KG']
        merged['Order Required KG'] = merged['Stock Balance'].apply(lambda x: max(0, -x) if pd.notna(x) else 0)

        # 5. Calculate Bobbins Required
        def calculate_bobbins(row):
            # Use .get() with a default value for 'Frame Needed' to avoid KeyError if merge failed for this row
            frame_needed = row.get('Frame Needed')
            if pd.isna(frame_needed) or row['Planned KG'] <= 0:
                return 0
            return math.ceil(row['Planned KG'] / (2600 if frame_needed == 2 else 1300))

        merged['Bobbins Required'] = merged.apply(calculate_bobbins, axis=1)

        # Return final columns - ensure all output_cols exist in merged
        output_cols = [
            'Prod. Order No.', 'Item Description', 'Clean Description',
            'Frame Needed', 'Planned KG', 'Stock',
            'Stock Balance', 'Order Required KG', 'Bobbins Required'
        ]
        # Filter output_cols to only include columns present in merged
        output_cols_present = [col for col in output_cols if col in merged.columns]


        # Create summary dictionary
        summary = {
            'single_frame_items': single_frame_count,
            'double_frame_items': double_frame_count,
            'total_bobbins': merged['Bobbins Required'].sum(),
            'result_df': merged[output_cols_present]
        }

        return summary

    except Exception as e:
        print(f"Error processing files: {str(e)}")
        return {'error': str(e)}

# Usage
result = calculate_bobbin_plan('bw plan (1).xlsx', 'material wise planning.xlsx')

if 'error' not in result:
    print(f"Single Frame Items: {result['single_frame_items']}")
    print(f"Double Frame Items: {result['double_frame_items']}")
    print(f"Total Bobbins Required: {result['total_bobbins']}")

    display(result['result_df'])
    result['result_df'].to_excel('bobbin_plan_output.xlsx', index=False)
else:
    print(f"Error occurred: {result['error']}")

ModuleNotFoundError: No module named 'fuzzywuzzy'

In [ ]:
pip install fuzzywuzzy


In [ ]:
print(result)

{'single_frame_items': np.int64(1), 'double_frame_items': np.int64(735), 'total_bobbins': np.int64(3278), 'result_df':           Prod. Order No.                  Item Description  \
0      WW/ALPRO/25-26/011       WOOL COTTON WHITE-DW-4.20/3   
1      WW/ALPRO/25-26/015       WOOL COTTON WHITE-DW-4.20/3   
2      WW/ALPRO/25-26/016       WOOL COTTON WHITE-DW-4.20/3   
3      WW/ALPRO/25-26/017       WOOL COTTON WHITE-DW-4.20/3   
4      WW/SAM/24-25/00274       WOOL COTTON WHITE-DW-4.20/3   
...                   ...                               ...   
2692  WWPL/PRO/25-26/0018  WOOL 5F- WHITE STRIA -DW- 12/2x3   
2693  WWPL/PRO/25-26/0133  WOOL 5F- WHITE STRIA -DW- 12/2x3   
2694  WWPL/PRO/25-26/0134  WOOL 5F- WHITE STRIA -DW- 12/2x3   
2695  WWPL/PRO/25-26/0589  WOOL 5F- WHITE STRIA -DW- 12/2x3   
2696  WWPL/PRO/25-26/0590  WOOL 5F- WHITE STRIA -DW- 12/2x3   

             Clean Description  Frame Needed    Planned KG     Stock  \
0         WOOL COTTON WHITE-DW           2.0  10118.